In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ============================
# 1. Setup: download FastText
# ============================
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz   # extracts cc.en.300.bin

--2025-11-13 08:38:56--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.225.143.54, 13.225.143.109, 13.225.143.99, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.225.143.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   276MB/s    in 14s     

2025-11-13 08:39:10 (311 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [2]:
import fasttext
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
from datasets import load_dataset
import itertools
from tqdm import tqdm
import json
import os

# -------------------
# 0. Create checkpoint folders (PHASE 1)
# -------------------
os.makedirs("checkpoints/phase1/best", exist_ok=True)
os.makedirs("checkpoints/phase1/final", exist_ok=True)

# -------------------
# 1. Load pretrained fastText
# -------------------
print("Loading fastText model...")
model = fasttext.load_model("cc.en.300.bin")

def get_embedding(text):
    return model.get_sentence_vector(str(text))

# -------------------
# 2. Aspects + results
# -------------------
aspects = ["appearance", "aroma", "palate", "taste"]
results = {}

# -------------------
# 3. Hyperparameters (grid-search)
# -------------------
param_grid = {
    "max_depth": [3, 5],
    "eta": [0.05, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1, 3],
    "num_boost_round": [3000]
}

# -------------------
# 4. Train each aspect
# -------------------
for aspect in aspects:
    print(f"\n========== Training Phase1 model for: {aspect} ==========")

    # Load dataset
    dataset = load_dataset("lengocquangLAB/beer-com-sentences")

    # Remove null rows
    def not_null(ex):
        return ex["reviewSentence"] is not None and ex[aspect] is not None

    dataset = dataset.filter(not_null)

    # -------------------
    # Build features
    # -------------------
    def build_features(ds, aspect):
        X, y = [], []
        for row in tqdm(ds, desc=f"Embedding {aspect}"):
            vec = get_embedding(row["reviewSentence"])
            X.append(vec)
            y.append(int(row[aspect]))  # 0/1 label
        return np.array(X), np.array(y)

    X_train, y_train = build_features(dataset["train"], aspect)
    X_valid, y_valid = build_features(dataset["validation"], aspect)
    X_test,  y_test  = build_features(dataset["test"], aspect)

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    best_params = None
    best_model = None
    best_f1 = -1

    print(f"Searching best hyperparameters for {aspect}...")

    param_combos = list(itertools.product(*param_grid.values()))

    for values in tqdm(param_combos, desc=f"Phase1 grid-search - {aspect}"):
        params = dict(zip(param_grid.keys(), values))
        num_boost_round = params.pop("num_boost_round")

        params.update({
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "reg_lambda": 1.0,
            "reg_alpha": 0.1
        })

        bst = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=num_boost_round,
            evals=[(dvalid, "valid")],
            early_stopping_rounds=300,
            verbose_eval=False
        )

        y_pred_val = (bst.predict(dvalid) > 0.5).astype(int)
        f1 = f1_score(y_valid, y_pred_val, average="macro")

        if f1 > best_f1:
            best_f1 = f1
            best_params = params
            best_model = bst

            print(f"New BEST F1 = {f1:.4f} for {aspect}")

            # 🔥 Save best checkpoint
            best_model.save_model(f"checkpoints/phase1/best/{aspect}_best_model.json")

    print(f"\nBest params for {aspect}: {best_params}")
    print(f"Best validation F1 for {aspect}: {best_f1:.4f}")

    # -------------------
    # Retrain full model (train + validation)
    # -------------------
    print(f"\nRetraining FINAL model for {aspect}...")

    X_train_full = np.vstack([X_train, X_valid])
    y_train_full = np.concatenate([y_train, y_valid])

    dtrain_full = xgb.DMatrix(X_train_full, label=y_train_full)
    dtest = xgb.DMatrix(X_test, label=y_test)

    final_model = xgb.train(
        params=best_params,
        dtrain=dtrain_full,
        num_boost_round=2000,
        evals=[(dtest, "test")],
        early_stopping_rounds=200,
        verbose_eval=False
    )

    # 🔥 Save final retrained model
    final_model.save_model(f"checkpoints/phase1/final/{aspect}_final_model.json")

    # -------------------
    # Evaluate
    # -------------------
    y_pred = (final_model.predict(dtest) > 0.5).astype(int)

    acc = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average="macro"
    )

    results[aspect] = {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "best_params": best_params
    }

    print(f"\n=== Final Test Evaluation for {aspect} ===")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print(classification_report(y_test, y_pred, digits=4))

# -------------------
# Save Phase1 Hyperparameters
# -------------------
with open("phase1_best_hyperparams.json", "w") as f:
    json.dump(results, f, indent=4)

print("\nAll Phase1 checkpoints saved in:")
print("  checkpoints/phase1/best/")
print("  checkpoints/phase1/final/")
print("Hyperparameters saved → phase1_best_hyperparams.json")

Loading fastText model...

========== Training Phase1 model for: appearance ==========


README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/663k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/93.4k [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

...

New BEST F1 = 0.8589 for taste


Phase1 grid-search - taste:  59%|█████▉    | 19/32 [04:41<04:22, 20.22s/it]

New BEST F1 = 0.8599 for taste


Phase1 grid-search - taste: 100%|██████████| 32/32 [08:54<00:00, 16.70s/it]



Best params for taste: {'max_depth': 5, 'eta': 0.05, 'subsample': 0.8, 'colsample_bytree': 1.0, 'min_child_weight': 1, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'reg_lambda': 1.0, 'reg_alpha': 0.1}
Best validation F1 for taste: 0.8599

Retraining FINAL model for taste...

=== Final Test Evaluation for taste ===
Accuracy:  0.9082
Precision: 0.8976
Recall:    0.8583
F1-score:  0.8753
              precision    recall  f1-score   support

           0     0.9168    0.9630    0.9393      1190
           1     0.8785    0.7536    0.8112       422

    accuracy                         0.9082      1612
   macro avg     0.8976    0.8583    0.8753      1612
weighted avg     0.9068    0.9082    0.9058      1612


All Phase1 checkpoints saved in:
  checkpoints/phase1/best/
  checkpoints/phase1/final/
Hyperparameters saved → phase1_best_hyperparams.json
